In [1]:
# Jovian Commit Essentials
# Please retain and execute this cell without modifying the contents for `jovian.commit` to work
!pip install jovian --upgrade -q
import jovian
jovian.set_project('linear-regression')
jovian.set_colab_id('1AKo4_TmdybnZAUVSmZJGvEX3Vov2WEqL')

     |████████████████████████████████| 71kB 3.6MB/s 


In [2]:
!pip install jovian --upgrade --quiet

In [3]:
import numpy as np
import torch

## Training data

We can represent the training data using two matrices: `inputs` and `targets`, each with one row per observation, and one column per variable.

In [4]:
# Input (temp, rainfall, humidity)
inputs = np.array([[73, 67, 43], 
                   [91, 88, 64], 
                   [87, 134, 58], 
                   [102, 43, 37], 
                   [69, 96, 70]], dtype='float32')

In [5]:
# Targets (apples, oranges)
targets = np.array([[56, 70], 
                    [81, 101], 
                    [119, 133], 
                    [22, 37], 
                    [103, 119]], dtype='float32')

Let's convert the arrays to PyTorch tensors.

In [6]:
inputs = torch.from_numpy(inputs)
targets = torch.from_numpy(targets)

## Linear regression model from scratch

The weights and biases (`w11, w12,... w23, b1 & b2`) can also be represented as matrices, initialized as random values. The first row of `w` and the first element of `b` are used to predict the first target variable, i.e., yield of apples, and similarly, the second for oranges.

yield_apple  = w11 * temp + w12 * rainfall + w13 * humidity + b1
yield_orange = w21 * temp + w22 * rainfall + w23 * humidity + b2

In [7]:
# Weights and biases
w = torch.randn((2, 3), requires_grad=True)
b = torch.randn((2,), requires_grad=True)
print(w)
print(b)

tensor([[ 0.5376,  0.0660, -0.4016],
        [-0.6218, -0.9119, -0.3167]], requires_grad=True)
tensor([-0.6848,  0.0488], requires_grad=True)


Our *model* is simply a function that performs a matrix multiplication of the `inputs` and the weights `w` (transposed) and adds the bias `b` (replicated for each observation).

# $$X \times w^T + b$$

We can define the model as follows:

In [8]:
def model(x):
  return torch.matmul(x, w.t()) + b

In [9]:
print(inputs)
print(w)
print(b)

tensor([[ 73.,  67.,  43.],
        [ 91.,  88.,  64.],
        [ 87., 134.,  58.],
        [102.,  43.,  37.],
        [ 69.,  96.,  70.]])
tensor([[ 0.5376,  0.0660, -0.4016],
        [-0.6218, -0.9119, -0.3167]], requires_grad=True)
tensor([-0.6848,  0.0488], requires_grad=True)


In [10]:
# Generate predictions
preds = model(inputs)
print(preds)
print('=' * 20)
print(targets)

tensor([[  25.7127, -120.0610],
        [  28.3416, -157.0552],
        [  31.6379, -194.6161],
        [  42.1283, -114.3067],
        [  14.6328, -152.5713]], grad_fn=<AddBackward0>)
tensor([[ 56.,  70.],
        [ 81., 101.],
        [119., 133.],
        [ 22.,  37.],
        [103., 119.]])


You can see a big difference between our model's predictions and the actual targets because we've initialized our model with random weights and biases. Obviously, we can't expect a randomly initialized model to just work.

## Loss function

Before we improve our model, we need a way to evaluate how well our model is performing. We can compare the model's predictions with the actual targets using the following method:

* Calculate the difference between the two matrices (`preds` and `targets`).
* Square all elements of the difference matrix to remove negative values.
* Calculate the average of the elements in the resulting matrix.

The result is a single number, known as the **mean squared error** (MSE).

In [11]:
# MSE loss
def mse(t1, t2):
  diff = t1-t2
  return torch.sum(diff * diff) / diff.numel()

In [12]:
loss = mse(preds, targets)
print(loss)

tensor(32622.8906, grad_fn=<DivBackward0>)


In [13]:
# diff = (preds - targets)**2
# loss = torch.mean(diff)
# loss

Here’s how we can interpret the result: On average, each element in the prediction differs from the actual target by the square root of the loss. And that’s pretty bad, considering the numbers we are trying to predict are themselves in the range 50–200. The result is called the loss because it indicates how bad the model is at predicting the target variables. It represents information loss in the model: the lower the loss, the better the model.

## Compute gradients
With PyTorch, we can automatically compute the gradient or derivative of the loss w.r.t. to the weights and biases because they have `requires_grad` set to `True`. We'll see how this is useful in just a moment.

In [14]:
# Compute gradients
loss.backward()

The gradients are stored in the `.grad` property of the respective tensors. Note that the derivative of the loss w.r.t. the weights matrix is itself a matrix with the same dimensions.

In [15]:
print(w)
print(w.grad)

tensor([[ 0.5376,  0.0660, -0.4016],
        [-0.6218, -0.9119, -0.3167]], requires_grad=True)
tensor([[ -3729.5278,  -5197.4883,  -3036.0898],
        [-20006.3574, -22384.1055, -13659.6445]])


In [16]:
import jovian

jovian

<module 'jovian' from '/usr/local/lib/python3.7/dist-packages/jovian/__init__.py'>

## Adjust weights and biases to reduce the loss
The increase or decrease in the loss by changing a weight element is proportional to the gradient of the loss w.r.t. that element. This observation forms the basis of _the gradient descent_ optimization algorithm that we'll use to improve our model (by _descending_ along the _gradient_).

We can subtract from each weight element a small quantity proportional to the derivative of the loss w.r.t. that element to reduce the loss slightly.

In [17]:
print(w)
print(w.grad)

tensor([[ 0.5376,  0.0660, -0.4016],
        [-0.6218, -0.9119, -0.3167]], requires_grad=True)
tensor([[ -3729.5278,  -5197.4883,  -3036.0898],
        [-20006.3574, -22384.1055, -13659.6445]])


In [18]:
# Let's take learning rate lr = 1e-5
lr = 1e-5
with torch.no_grad():
  w -= w.grad * lr
  b -= b.grad * lr

In [20]:
print(w)
print(b)

tensor([[ 0.5749,  0.1180, -0.3713],
        [-0.4217, -0.6881, -0.1801]], requires_grad=True)
tensor([-0.6843,  0.0512], requires_grad=True)


We multiply the gradients with a very small number ($10^{-5}$ in this case) to ensure that we don't modify the weights by a very large amount. We want to take a small step in the downhill direction of the gradient, not a giant leap. This number is called the *learning rate* of the algorithm. 

We use `torch.no_grad` to indicate to PyTorch that we shouldn't track, calculate, or modify gradients while updating the weights and biases.  

Before we proceed to verify that the loss is actually lower, we reset the gradients to zero by invoking the `.zero_()` method. We need to do this because PyTorch accumulates gradients. Otherwise, the next time we invoke `.backward` on the loss, the new gradient values are added to the existing gradients, which may lead to unexpected results.

In [22]:
w.grad.zero_()
b.grad.zero_()
print(w.grad)
print(b.grad)

tensor([[0., 0., 0.],
        [0., 0., 0.]])
tensor([0., 0.])


## Train the model using gradient descent

As seen above, we reduce the loss and improve our model using the gradient descent optimization algorithm. Thus, we can _train_ the model using the following steps:

1. Generate predictions

2. Calculate the loss

3. Compute gradients w.r.t the weights and biases

4. Adjust the weights by subtracting a small quantity proportional to the gradient

5. Reset the gradients to zero

Let's implement the above step by step.

In [23]:
# Generate predictions
preds = model(inputs)
print(preds)

tensor([[  33.2236,  -84.5830],
        [  38.2528, -110.4068],
        [  43.6087, -139.2908],
        [  49.2912,  -79.2186],
        [  24.3215, -107.7140]], grad_fn=<AddBackward0>)


In [24]:
# Calculate the loss
loss = mse(preds, targets)
print(loss)

tensor(22260.2070, grad_fn=<DivBackward0>)


In [25]:
# Compute gradients
loss.backward()
print(w.grad)
print(b.grad)

tensor([[ -2951.3674,  -4353.9653,  -2517.1250],
        [-16341.8887, -18441.9551, -11228.0078]])
tensor([ -38.4605, -196.2426])


Let's update the weights and biases using the gradients computed above.

In [26]:
# Adjust weights & reset gradients
with torch.no_grad():
  w -= w.grad * lr
  b -= b.grad * lr
  w.grad.zero_()
  b.grad.zero_()

Let's take a look at the new weights and biases.

In [27]:
print(w)
print(b)

tensor([[ 0.6044,  0.1615, -0.3461],
        [-0.2583, -0.5037, -0.0679]], requires_grad=True)
tensor([-0.6840,  0.0531], requires_grad=True)


With the new weights and biases, the model should have a lower loss.

In [28]:
# Calculate loss
print(f"loss before: {loss}")
preds = model(inputs)
loss = mse(preds, targets)
print(f"loss after: {loss}")

loss before: 22260.20703125
loss after: 15273.6279296875


## Train for multiple epochs

To reduce the loss further, we can repeat the process of adjusting the weights and biases using the gradients multiple times. Each iteration is called an _epoch_. Let's train the model for 100 epochs.

In [30]:
for i in range(100):
  # generate predictions
  preds = model(inputs)
  # calculate the loss
  loss = mse(preds, targets)
  # compute gradients
  loss.backward()
  # adjust the weights and reset the gradients
  with torch.no_grad():
      w -= w.grad * lr
      b -= b.grad * lr
      w.grad.zero_()
      b.grad.zero_()


Once again, let's verify that the loss is now lower:

In [32]:
# Calculate loss
preds = model(inputs)
loss = mse(preds, targets)
print(f"loss= {loss}")

loss= 267.3872375488281


The loss is now much lower than its initial value. Let's look at the model's predictions and compare them with the targets.

In [33]:
# predictions
print(preds)

print("=" * 50)

# targets
print(targets)


tensor([[ 63.0731,  74.6333],
        [ 80.7282, 101.0593],
        [112.5561, 125.1219],
        [ 55.5345,  61.8349],
        [ 79.1074, 105.3428]], grad_fn=<AddBackward0>)
tensor([[ 56.,  70.],
        [ 81., 101.],
        [119., 133.],
        [ 22.,  37.],
        [103., 119.]])


The predictions are now quite close to the target variables. We can get even better results by training for a few more epochs. 

In [34]:
jovian.commit()

[jovian] Detected Colab notebook...
[jovian] Please enter your API key ( from https://jovian.ai/ ):
API KEY: ··········
[jovian] Uploading colab notebook to Jovian...
[jovian] Capturing environment..
[jovian] Committed successfully! https://jovian.ai/boubekri/linear-regression


'https://jovian.ai/boubekri/linear-regression'